In [ ]:
%run "standardScalerSetup.ipynb"
%run "minMaxScalerSetup.ipynb"
#loads all the preprocessing libraries and prepares the dataframe

In [ ]:
from sklearn import metrics

In [ ]:
#LSTM models
# input shape format is [samples, timesteps, features]
# base has [30, 1, 9]

In [ ]:
def format_to_lstm(df):
    X = np.array(df)
    return np.reshape(X, (X.shape[0], 1, X.shape[1]))

In [ ]:
# Transform the datasets for LSTM processing
lstm_X_test1_summaryL1Z2_std = format_to_lstm(X_test1_summaryL1Z2_std)
lstm_X_test1_summaryL1Z2_minMax = format_to_lstm(X_test1_summaryL1Z2_minMax)
lstm_X_train_base_std = format_to_lstm(X_train_base_std)
lstm_X_train_base_minMax = format_to_lstm(X_train_base_minMax)
lstm_X_train_all_std = format_to_lstm(X_train_all_std)
lstm_X_train_all_minMax = format_to_lstm(X_train_all_minMax)
lstm_X_test_all_std = format_to_lstm(X_test_all_std)
lstm_X_test_all_minMax = format_to_lstm(X_test_all_minMax)

In [ ]:
lstm_X_test1_summaryL1Z2_std.shape

In [ ]:
lstm_X_test_all_std.shape

In [ ]:
lstm_X_train_base_std.shape

In [ ]:
# Neural Nets imports
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
#Define LSTM model
#input shape for the LSTM is (number_steps x number_features)

def lstm_model_mse(layerSize, numSteps, numFeatures):
    
    model = Sequential()
    model.add(LSTM(layerSize, return_sequences=True, input_shape=(numSteps,numFeatures),
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                   dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(layerSize, input_shape=(numSteps,numFeatures), 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                   dropout=0.2, recurrent_dropout=0.2))

    model.add(Dense(layerSize, kernel_initializer='normal',
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation=custom_activation))
    #model.add(BatchNormalization())
    
    model.add(Dense(1, kernel_initializer='normal', 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation='linear'))

    model.compile(Adam(lr=0.001),
              loss='mse',
              metrics=['mse'])
    return model

def lstm_model_mae(layerSize, numSteps, numFeatures):
    
    model = Sequential()
    model.add(LSTM(layerSize, return_sequences=True, input_shape=(numSteps,numFeatures),
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                   dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(layerSize, input_shape=(numSteps,numFeatures), 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                   dropout=0.2, recurrent_dropout=0.2))

    model.add(Dense(layerSize, kernel_initializer='normal',
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation=custom_activation))
    #model.add(BatchNormalization())
    
    model.add(Dense(1, kernel_initializer='normal', 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation='linear'))
    
    model.compile(Adam(lr=0.001),
              loss='mae',
              metrics=['mae'])
    return model

In [ ]:
verboseLevel=0
validationSplit=0.2
batchSize=30
epochs=1000

In [ ]:
# callback preparation
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)

In [ ]:
checkpoint_lstm_std_base_mae = ModelCheckpoint("lstm_model_std_base_mae_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_std_base_mae = [checkpoint_lstm_std_base_mae, reduce_lr]

checkpoint_lstm_std_base_mse = ModelCheckpoint("lstm_model_std_base_mse_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_std_base_mse = [checkpoint_lstm_std_base_mse, reduce_lr]

In [ ]:
layerSize = 64
numSteps = 1
numFeatures = 9

In [ ]:
lstm_std_base_mae_model = lstm_model_mae(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)

In [ ]:
lstm_std_base_mae_model.summary()

In [ ]:
lstm_std_base_mse_model = lstm_model_mse(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)

In [ ]:
%%time
history_LSTM_std_base_mse = lstm_std_base_mse_model.fit(lstm_X_train_base_std, y_train_scaled_std_base,
                                batch_size=5, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_std_base_mse)

In [ ]:
%%time
history_LSTM_std_base_mae = lstm_std_base_mae_model.fit(lstm_X_train_base_std, y_train_scaled_std_base,
                                batch_size=5, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_std_base_mae)

In [ ]:
##################################################
# LSTM STD BASE MAE
#################################################
#Reload the best saved model
lstm_std_base_mae_model_new = load_model('lstm_model_std_base_mae_L1_L2.h5')
#Predict
y_pred_LSTM_base_std_L1Z2_MAE = lstm_std_base_mae_model_new.predict(lstm_X_test1_summaryL1Z2_std)
pred = targetStdScalerBase.inverse_transform(y_pred_LSTM_base_std_L1Z2_MAE)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_std, pred))

In [ ]:
minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred)
minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_std, pred)
modelNameAWS_base = "LSTM_STD_BASE_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_std_base_mae.history['loss'])
plt.plot(history_LSTM_std_base_mae.history['val_loss'])
plt.title('LSTM MAE STD ALL model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_std.shape[0]),y_test1_summaryL1Z2_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM1 MAE Std Base Model for L1Z2_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
##################################################
# LSTM STD BASE MSE
#################################################
#Reload the best saved model
lstm_std_base_mse_model_new = load_model('lstm_model_std_base_mse_L1_L2.h5')
#Predict
y_pred_LSTM_base_std_L1Z2_MSE = lstm_std_base_mse_model_new.predict(lstm_X_test1_summaryL1Z2_std)
pred = targetStdScalerBase.inverse_transform(y_pred_LSTM_base_std_L1Z2_MSE)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_std, pred))

In [ ]:
if metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred) < minMaeAWS_base:
    minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred)
    minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_std, pred)
    modelNameAWS_base = "LSTM_STD_BASE_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_std_base_mse.history['loss'])
plt.plot(history_LSTM_std_base_mse.history['val_loss'])
plt.title('LSTM MSE STD ALL model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_std.shape[0]),y_test1_summaryL1Z2_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM1 MSE Std Base Model for L1Z2_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
#MinMax Base

In [ ]:
checkpoint_lstm_minMax_base_mae = ModelCheckpoint("lstm_model_minMax_base_mae_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_minMax_base_mae = [checkpoint_lstm_minMax_base_mae, reduce_lr]

checkpoint_lstm_minMax_base_mse = ModelCheckpoint("lstm_model_minMax_base_mse_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_minMax_base_mse = [checkpoint_lstm_minMax_base_mse, reduce_lr]

In [ ]:
lstm_minMax_base_mae_model = lstm_model_mae(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)
lstm_minMax_base_mse_model = lstm_model_mse(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)

In [ ]:
%%time
history_LSTM_minMax_base_mae = lstm_minMax_base_mae_model.fit(lstm_X_train_base_minMax, y_train_scaled_minMax_base,
                                batch_size=5, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_minMax_base_mae)

In [ ]:
%%time
history_LSTM_minMax_base_mse = lstm_minMax_base_mse_model.fit(lstm_X_train_base_minMax, y_train_scaled_minMax_base,
                                batch_size=5, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_minMax_base_mse)

In [ ]:
##################################################
# LSTM MinMax BASE MAE
#################################################
#Reload the best saved model
lstm_minMax_base_mae_model_new = load_model('lstm_model_minMax_base_mae_L1_L2.h5')
#Predict
y_pred_LSTM_base_minMax_L1Z2_MAE = lstm_minMax_base_mae_model_new.predict(lstm_X_test1_summaryL1Z2_minMax)
pred = targetMinMaxScalerBase.inverse_transform(y_pred_LSTM_base_minMax_L1Z2_MAE)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred) < minMaeAWS_base:
    minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred)
    minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_minMax, pred)
    modelNameAWS_base = "LSTM_MinMax_BASE_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_minMax_base_mae.history['loss'])
plt.plot(history_LSTM_minMax_base_mae.history['val_loss'])
plt.title('LSTM MAE MinMax ALL model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_minMax.shape[0]),y_test1_summaryL1Z2_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM1 MAE MinMax Base Model for L1Z2_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
##################################################
# LSTM MinMax BASE MSE
#################################################
#Reload the best saved model
lstm_minMax_base_mse_model_new = load_model('lstm_model_minMax_base_mse_L1_L2.h5')
#Predict
y_pred_LSTM_base_minMax_L1Z2_MSE = lstm_minMax_base_mse_model_new.predict(lstm_X_test1_summaryL1Z2_minMax)
pred = targetMinMaxScalerBase.inverse_transform(y_pred_LSTM_base_minMax_L1Z2_MSE)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred) < minMaeAWS_base:
    minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred)
    minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_minMax, pred)
    modelNameAWS_base = "LSTM_MinMax_BASE_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_minMax_base_mse.history['loss'])
plt.plot(history_LSTM_minMax_base_mse.history['val_loss'])
plt.title('LSTM MSE MinMax Base model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_minMax.shape[0]),y_test1_summaryL1Z2_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM1 L2 MinMax Base Model for L1Z2_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
# STD ALL

In [ ]:
checkpoint_lstm_std_all_mae = ModelCheckpoint("lstm_model_std_all_mae_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_std_all_mae = [checkpoint_lstm_std_all_mae, reduce_lr]

checkpoint_lstm_std_all_mse = ModelCheckpoint("lstm_model_std_all_mse_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_std_all_base_mse = [checkpoint_lstm_std_all_mse, reduce_lr]

In [ ]:
lstm_std_all_mae_model = lstm_model_mae(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)
lstm_std_all_mse_model = lstm_model_mse(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)

In [ ]:
%%time
history_LSTM_std_all_mae = lstm_std_all_mae_model.fit(lstm_X_train_all_std, y_train_scaled_std_all,
                                batch_size=batchSize, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_std_all_mae)

In [ ]:
%%time
history_LSTM_std_all_mse = lstm_std_all_mse_model.fit(lstm_X_train_all_std, y_train_scaled_std_all,
                                batch_size=batchSize, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_std_all_base_mse)

In [ ]:
##################################################
# LSTM STD ALL MAE
#################################################
#Reload the best saved model
lstm_std_all_mae_model_new = load_model('lstm_model_std_all_mae_L1_L2.h5')
#Predict
y_pred_LSTM_std_all_mae = lstm_std_all_mae_model_new.predict(lstm_X_test_all_std)
pred = targetStdScalerAll.inverse_transform(y_pred_LSTM_std_all_mae)
#Show results
print(metrics.r2_score(y_test_all_std, pred))
print(metrics.mean_absolute_error(y_test_all_std, pred))
print(metrics.mean_squared_error(y_test_all_std, pred))

In [ ]:
minMaeAWS = metrics.mean_absolute_error(y_test_all_std, pred)
minR2AWS = metrics.r2_score(y_test_all_std, pred)
modelNameAWS = "LSTM_STD_ALL_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_std_all_mae.history['loss'])
plt.plot(history_LSTM_std_all_mae.history['val_loss'])
plt.title('LSTM MAE STD ALL model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_std.shape[0]),y_test_all_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM1 MAE STD ALL Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
##################################################
# LSTM STD ALL MSE
#################################################
#Reload the best saved model
lstm_std_all_mse_model_new = load_model('lstm_model_std_all_mse_L1_L2.h5')
#Predict
y_pred_LSTM_std_all_mse = lstm_std_all_mse_model_new.predict(lstm_X_test_all_std)
pred = targetStdScalerAll.inverse_transform(y_pred_LSTM_std_all_mse)
#Show results
print(metrics.r2_score(y_test_all_std, pred))
print(metrics.mean_absolute_error(y_test_all_std, pred))
print(metrics.mean_squared_error(y_test_all_std, pred))

In [ ]:
if metrics.mean_absolute_error(y_test_all_std, pred) < minMaeAWS:
    minMaeAWS = metrics.mean_absolute_error(y_test_all_std, pred)
    minR2AWS = metrics.r2_score(y_test_all_std, pred)
    modelNameAWS = "LSTM_STD_ALL_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_std_all_mse.history['loss'])
plt.plot(history_LSTM_std_all_mse.history['val_loss'])
plt.title('LSTM MSE STD ALL model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_std.shape[0]),y_test_all_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM1 MSE STD ALL Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
#MinMax All

In [ ]:
checkpoint_lstm_minMax_all_mae = ModelCheckpoint("lstm_model_minMax_all_mae_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_minMax_all_mae = [checkpoint_lstm_minMax_all_mae, reduce_lr]

checkpoint_lstm_minMax_all_mse = ModelCheckpoint("lstm_model_minMax_all_mse_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_minMax_all_mse = [checkpoint_lstm_minMax_all_mse, reduce_lr]

In [ ]:
lstm_minMax_all_mae_model = lstm_model_mae(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)
lstm_minMax_all_mse_model = lstm_model_mse(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)

In [ ]:
%%time
history_LSTM_minMax_all_mae = lstm_minMax_all_mae_model.fit(lstm_X_train_all_minMax, y_train_scaled_minMax_all,
                                batch_size=batchSize, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_minMax_all_mae)

In [ ]:
%%time
history_LSTM_minMax_all_mse = lstm_minMax_all_mse_model.fit(lstm_X_train_all_minMax, y_train_scaled_minMax_all,
                                batch_size=batchSize, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_minMax_all_mse)

In [ ]:
##################################################
# LSTM MinMax ALL MAE
#################################################
#Reload the best saved model
lstm_minMax_all_mae_model_new = load_model('lstm_model_minMax_all_mae_L1_L2.h5')
#Predict
y_pred_LSTM_minMax_all_mae = lstm_minMax_all_mae_model_new.predict(lstm_X_test_all_minMax)
pred = targetMinMaxScalerAll.inverse_transform(y_pred_LSTM_minMax_all_mae)
#Show results
print(metrics.r2_score(y_test_all_minMax, pred))
print(metrics.mean_absolute_error(y_test_all_minMax, pred))
print(metrics.mean_squared_error(y_test_all_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test_all_minMax, pred) < minMaeAWS:
    minMaeAWS = metrics.mean_absolute_error(y_test_all_minMax, pred)
    minR2AWS = metrics.r2_score(y_test_all_minMax, pred)
    modelNameAWS = "LSTM_MinMax_ALL_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_minMax_all_mae.history['loss'])
plt.plot(history_LSTM_minMax_all_mae.history['val_loss'])
plt.title('LSTM L1 MinMax ALL model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_minMax.shape[0]),y_test_all_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM MAE MinMax ALL Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
##################################################
# LSTM MinMax ALL MSE
#################################################
#Reload the best saved model
lstm_minMax_all_mse_model_new = load_model('lstm_model_minMax_all_mse_L1_L2.h5')
#Predict
y_pred_LSTM_minMax_all_mse = lstm_minMax_all_mse_model_new.predict(lstm_X_test_all_minMax)
pred = targetMinMaxScalerAll.inverse_transform(y_pred_LSTM_minMax_all_mse)
#Show results
print(metrics.r2_score(y_test_all_minMax, pred))
print(metrics.mean_absolute_error(y_test_all_minMax, pred))
print(metrics.mean_squared_error(y_test_all_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test_all_minMax, pred) < minMaeAWS:
    minMaeAWS = metrics.mean_absolute_error(y_test_all_minMax, pred)
    minR2AWS = metrics.r2_score(y_test_all_minMax, pred)
    modelNameAWS = "LSTM_MinMax_ALL_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM_minMax_all_mse.history['loss'])
plt.plot(history_LSTM_minMax_all_mse.history['val_loss'])
plt.title('LSTM MSE MinMax ALL model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_minMax.shape[0]),y_test_all_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM MSE MinMax ALL Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
minMaeAWS_base, minR2AWS_base, modelNameAWS_base

In [ ]:
minMaeAWS, minR2AWS, modelNameAWS

In [ ]:
#### END ####

In [ ]:
# The below is how to create multivariate time series with multiple steps from a single step multivariate series
def format_to_lstm_nsteps(df_X, df_Y, n_steps=1):
    X, y = list(), list()
    df_X_a = df_X.to_numpy()
    #df_Y_a = df_Y.to_numpy()
    for i in range(len(df_X)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(df_X):
            break
        # gather input and output parts of the pattern
        seq_x = df_X_a[i:end_ix, :]
        seq_y = df_Y[end_ix-1]
        #print(f'Iteration: {i}, end_ix={end_ix}')
        #print(f'seq_x={seq_x}')
        #print(f'seq_y={seq_y}')
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
X_train_lstm3, y_train_lstm3 = format_to_lstm_nsteps(X_train_base_std, y_train_scaled_std_base,3)

In [ ]:
X_train_lstm3.shape

In [ ]:
y_train_lstm3.shape

In [ ]:
lstm3_std_base_mae_model = lstm_model_mae(layerSize=layerSize, numSteps=3, numFeatures=numFeatures)
lstm3_std_base_mse_model = lstm_model_mse(layerSize=layerSize, numSteps=3, numFeatures=numFeatures)

In [ ]:
lstm3_std_base_mae_model.summary()

In [ ]:
checkpoint_lstm3_std_base_mae = ModelCheckpoint("lstm3_model_std_base_mae_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm3_std_base_mae = [checkpoint_lstm3_std_base_mae, reduce_lr]

checkpoint_lstm3_std_base_mse = ModelCheckpoint("lstm3_model_std_base_mse_L1_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm3_std_base_mse = [checkpoint_lstm3_std_base_mse, reduce_lr]

In [ ]:
%%time
history_LSTM3_std_base_mae = lstm3_std_base_mae_model.fit(X_train_lstm3, y_train_lstm3,
                                batch_size=5, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm3_std_base_mae)

In [ ]:
%%time
history_LSTM3_std_base_mse = lstm3_std_base_mse_model.fit(X_train_lstm3, y_train_lstm3,
                                batch_size=5, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm3_std_base_mse)

In [ ]:
X_test_lstm3, y_test_lstm3 = format_to_lstm_nsteps(X_test1_summaryL1Z2_std, y_test_scaled_std_base, 3)

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM3_std_base_mae.history['loss'])
plt.plot(history_LSTM3_std_base_mae.history['val_loss'])
plt.title('LSTM3_std_base MAE model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#Reload the best saved model
lstm3_std_base_mae_model_new = load_model('lstm3_model_std_base_mae_L1_L2.h5')
#Predict
y_pred_LSTM3_base_std_L1Z2_mae = lstm3_std_base_mae_model_new.predict(X_test_lstm3)
pred = targetStdScalerBase.inverse_transform(y_pred_LSTM3_base_std_L1Z2_mae)
y_target = targetStdScalerBase.inverse_transform(y_test_lstm3)
#Show results
print(metrics.r2_score(y_target, pred))
print(metrics.mean_absolute_error(y_target, pred))
print(metrics.mean_squared_error(y_target, pred))

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_target.shape[0]),y_target,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM3 MAE Std Base Model for L1Z2_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_LSTM3_std_base_mse.history['loss'])
plt.plot(history_LSTM3_std_base_mse.history['val_loss'])
plt.title('LSTM3_std_base MSE model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#Reload the best saved model
lstm3_std_base_mse_model_new = load_model('lstm3_model_std_base_mse_L1_L2.h5')
#Predict
y_pred_LSTM_base_std_L1Z2_mse = lstm3_std_base_mse_model_new.predict(X_test_lstm3)
pred = targetStdScalerBase.inverse_transform(y_pred_LSTM_base_std_L1Z2_mse)
y_target = targetStdScalerBase.inverse_transform(y_test_lstm3)
#Show results
print(metrics.r2_score(y_target, pred))
print(metrics.mean_absolute_error(y_target, pred))
print(metrics.mean_squared_error(y_target, pred))

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_target.shape[0]),y_target,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('LSTM3 MSE Std Base Model for L1Z2_test dataset prediction vs original')
plt.legend()
plt.show()